In [1]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
#Fonction qui recupere les donnees des restaurants sur une page alloresto et les ajoute à une liste
def recuperer_liste_restaurants(link, adresse, liste_restaurants, liste_plats):
    
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # go to the page
    driver.get(link)

    #recuperer la liste des restaurants poour une adresse donnée
    inputSearch = driver.find_element_by_name("address_search")
    inputSearch.send_keys(adresse)
    inputSearch.send_keys(Keys.RETURN);
    
    # attendre que la page charge les restaurant supplémentaires
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'address-search--confirm')))
    
    #On va à la page des restaurants 
    inputElement = driver.find_element_by_css_selector("button.address-search--confirm")
    #print(inputElement)
    inputElement.click()
    # attendre que la page charge les restaurant supplémentaires
    WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'restaurant-index-page-tile')))    
    
    #req = requests.get(link)
    #codeSource = req.text
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    restaurants = soup.find_all('li', {"class" : "restaurant-index-page-tile"})
    
    #TODO: A retirer
    restaurants = restaurants[1:4]
    #print (restaurants)
    
    for restaurant in restaurants:
        type(restaurant)
        restaurant = str(restaurant) 
        soup_r = BeautifulSoup(restaurant, 'html.parser')

        #link
        link_r = 'https://deliveroo.fr/fr' + soup_r.find("a")["href"]

        #On vérifie que le restaurant n'est pas déjà dans la liste
        isInList = False
        for r in liste_restaurants:
            if r["link"]==link:
                isInList = True

        if not isInList:

            #id
            id_restaurant = soup_r.find('li')['data-reactid'].split("$")[-1]

            #name
            name = soup_r.find("h3").text.replace("\n","")
            
            #specialties
            specialties = soup_r.find("p").text.split(" ・ ")
            specialties = [x for x in specialties if "€" not in x]
            
            #logo
            logo = ""
            try:
                logo = soup_r.find("span", {"class": "restaurant-index-page-tile--top"})['style'].split("\"")[1]
            except:
                pass
            
            #On recupere les infos issues de la page du resto
            
            #Avec selenium
            #driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 't') 
            #driver.get(link_r)
            #codeSource = driver.page_source
            #driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'w') 
            
            #Sans selenium
            codeSource = requests.get(link_r).text
            soup_page_restaurant = BeautifulSoup(codeSource, 'html.parser')
            
            #address
            address = ""
            try:
                address = soup_page_restaurant.find("small", {"class": "address"}).text
            except:
                pass
            
            #phone
            phone = ""
            try:
                phone = soup_page_restaurant.find("small", {"class": "phone"}).text
            except:
                pass
            
            #description
            description = ""
            try:
                description = soup_page_restaurant.find("div", {"class": "restaurant__description"}).text
            except:
                pass
            
            #work_time
            work_time = ""
            try:
                work_time = soup_page_restaurant.find("small", {"class": "opening-hours"}).text
            except:
                pass

            #print(type(name))
            objet = {
                "id" : id_restaurant,
                "name" : name,
                "logo" : logo,
                "address" : address,
                "phone" : phone,
                "description" : description,
                "specialties" : specialties,
                "work_time" : work_time,
                #"minimum_order" : minimum_order,
                "link" : link_r
            }

            liste_restaurants.append(objet)
            #Appel de la fonction ajoutant la liste des plats
            recuperer_liste_plats(codeSource, id_restaurant, liste_plats)

    #print(i)
    driver.quit()

In [3]:
#recuperer les donnes sur les plats d'un restaurant donne
def recuperer_liste_plats(codeSource, id_restaurant, liste_plats):
    soup_carte = BeautifulSoup(codeSource, 'html.parser')
    plats = soup_carte.find_all("li", {"class" : "menu-index-page__item"})
    
    for plat in plats:
        soup_p = BeautifulSoup(str(plat) , 'html.parser')
        #print (soup_p.prettify())
        
        #id
        id_plat = soup_p.find("li")["data-reactid"].split("$")[-1]

        #On vérifie que le plat ajouté n'est pas deja dans la liste
        isInList = False
        for p in liste_plats:
            if p["id"]==id_plat:
                isInList = True

        if not isInList:
            #nom du plat
            name = soup_p.find("h6").text

            #logo
            logo = ""
            try:
                logo = soup_p.find("div", {"class" : "menu-index-page__item-image"})['style']\
                        .replace("background-image:url(", "http:")\
                        .replace(");","")
            except:
                pass
            
            #prix
            prix = soup_p.find("span" , {"class" : "menu-index-page__item-price"}).text
            prix = " ".join(prix.split())

            #description
            description = ""
            try:
                description = soup_p.find("p" , {"class" : "menu-index-page__item-desc"}).text
            except:
                pass                
                
            objet = {
                "id_restaurant" : id_restaurant,
                "id" : id_plat,
                "name" : name,
                "prix" : prix,
                "descrption" : description,
                "logo" : logo
            }

            liste_plats.append(objet)

In [4]:
liste_restaurants = []
liste_plats = []
link = "https://deliveroo.fr/fr/"
adresse="1 Avenue Daumesnil, 75012 Paris, France"
recuperer_liste_restaurants(link, adresse, liste_restaurants, liste_plats)

In [5]:
liste_restaurants

[{'address': '2 rue de la Verrerie, Paris, 75004',
  'description': '" Arrêtez de manger de la merde ! "Read more',
  'id': '13827',
  'link': 'https://deliveroo.fr/fr/menu/paris/4eme-hotel-de-ville/bagelstein-verrerie?day=today&rpos=1&time=ASAP',
  'logo': 'https://cdn1.deliveroo.co.uk/media/menus/12225/400x225.jpg?v=1474304974&quality=80',
  'name': 'Bagelstein Verrerie',
  'phone': '+33142710666',
  'specialties': ['Américain', 'Bagels'],
  'work_time': "Ouvert jusqu'à 23:00"},
 {'address': '1 rue des Francs Bourgeois, Paris, 75004',
  'description': 'Glace italienne d’Excellence, 100% issue de produits naturels et d’ingrédients sélectionnés, sans colorant, ni arôme artificiel. Découvrez nos coffrets isothermes et nos Macarons al Gelato.Read more',
  'id': '13861',
  'link': 'https://deliveroo.fr/fr/menu/paris/4eme-hotel-de-ville/amorino-francs-bourgeois?day=today&rpos=2&time=ASAP',
  'logo': 'https://cdn1.deliveroo.co.uk/media/menus/18565/400x225.jpg?v=1473408098&quality=80',
  'na

In [6]:
len(liste_restaurants)

3

In [7]:
len(liste_plats)

148

In [8]:
liste_plats

[{'descrption': 'Bagel aux graines de tournesol, cocktail de crevettes au fromage frais et persillade, mesclun, tomate, concombre, sauce yaourt citron',
  'id': '811936',
  'id_restaurant': '13827',
  'logo': '',
  'name': 'Bagel Du Mois',
  'prix': '7,20 €'},
 {'descrption': 'Bagel (au choix), cream cheese, salade, mozzarella, tomates séchées, poivrons marinés, sauce (au choix)',
  'id': '794767',
  'id_restaurant': '13827',
  'logo': '',
  'name': 'Isidore',
  'prix': '6,40 €'},
 {'descrption': 'Bagel (au choix), mousse de thon, salade, concombres, tomates, sauce (au choix)',
  'id': '794768',
  'id_restaurant': '13827',
  'logo': '',
  'name': 'Henriette',
  'prix': '6,40 €'},
 {'descrption': 'Bagel (au choix), cream cheese, salade, saumon fumé, concombres, oignons rouges, sauce (au choix)',
  'id': '794770',
  'id_restaurant': '13827',
  'logo': 'http://cdn1.deliveroo.co.uk/media/menu_items/794770/320x180.jpg?v=1479910945',
  'name': 'Léontine',
  'prix': '6,90 €'},
 {'descrption':